Here in this notebook, i will be trying to make python code that extracts and then summarize the content of a pdf file and then highlight the key phrases in the summary, as the pdf file could be harder to read as they aren't originally designed as text files.

This can helps in categorizing pdf files later when a researcher or an institute wants to classifiy the files for various reasons

Hier in diesem Notizbuch werde ich versuchen, Python-Code zu erstellen, der den Inhalt einer PDF-Datei extrahiert und zusammenfasst und dann die Schlüsselsätze in der Zusammenfassung hervorhebt, da die PDF-Datei schwerer zu lesen sein könnte, da sie ursprünglich nicht als Textdatei konzipiert ist.

Dies kann später beim Kategorisieren von PDF-Dateien hilfreich sein, wenn ein Forscher oder ein Institut die Dateien aus verschiedenen Gründen klassifizieren möchte.

In [1]:
# first i will install needed libraries
# PyMuPDF help in extracting texts from PDF files
# transformers is a "HuggingFace" library that include Summarization

# zuerst werde ich die benötigten Bibliotheken installieren
# PyMuPDF hilft beim Extrahieren von Texten aus PDF-Dateien
# transformers ist eine „HuggingFace“-Bibliothek, die Summarization enthält
!pip install PyMuPDF
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 81.1 MB/s eta 0:00:00


In [2]:
# Here i will import the needed libraries
# Hier werde ich die benötigten Bibliotheken importieren

# fitz is related to "PyMuPDF"
# fitz ist mit „PyMuPDF“ verwandt
import fitz
# and here i will import the part of "HuggingFace" transformation library used to create NLP piplines
# und hier werde ich den Teil der Transformationsbibliothek „HuggingFace“ importieren, der zum Erstellen von NLP-Pipelines verwendet wird
from transformers import pipeline

In [3]:
# Defining the function to extract text from a PDF file
# Definieren der Funktion zum Extrahieren von Text aus einer PDF-Datei

def extract_PDFtext(file_path):
    # create the variable that uses "fitz"
    # erstelle die Variable, die „fitz“ verwendet
    doc = fitz.open(file_path)
    # creating text, so that later i can add the extracted text to it
    # Text erstellen, damit ich später den extrahierten Text hinzufügen kann
    text = ""
    # iterate through each page in the document, "len" for giving the number of pages
    # durch jede Seite im Dokument iterieren, „len“ zur Angabe der Seitenzahl
    for page_num in range(len(doc)):
        # "load_page" is to the page specified by "page_num"
        # „load_page“ führt zu der durch „page_num“ angegebenen Seite
        page = doc.load_page(page_num)
        # den Inhalt einer Seite extrahieren
        text += page.get_text()
    # showing the result
    # zeigt das Ergebnis
    return text

In [4]:
# Getting the PDF file from the path
# PDF-Datei aus dem Pfad abrufen
PDF_path = "/content/S_Berry_Fashion_ed_Miller.pdf"

# Extracting the text from the PDF file
# Extrahieren des Textes aus der PDF-Datei
PDF_file_text = extract_PDFtext(PDF_path)
print("The Text of the PDF file: \n", PDF_file_text)

The Text of the PDF file: 
 See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/316404875
Fashion
Chapter · February 2008
DOI: 10.1002/9780470998809.ch26
CITATIONS
0
READS
2,919
1 author:
Sarah Berry
Western Governors University
4 PUBLICATIONS   7 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Sarah Berry on 15 January 2021.
The user has requested enhancement of the downloaded file.
ChaDter 26 
Fashion 
Sarah Berry 
Fashion is a tangential concern to most academic disciplines; outside of historical 
costume studies there has been little sustained attention to the topic. When 
approached within a traditional discipline, fashion is usually slotted into a 
broader theoretical framework rather than addressed with any conceptual speci- 
ficity. The relative paucity of research on fashion can be easily attributed to its 
association with femininity and trivial consumerism, but it is also a tremendously 
diffi

In [5]:
# the text is too big to be summarized, so i will first try to divided to chunks
# and i will start by seeing how many words is there in the text
# der Text ist zu lang, um ihn zusammenzufassen, also werde ich zunächst versuchen, ihn in Abschnitte aufzuteilen
# und ich werde zunächst sehen, wie viele Wörter der Text hat

def count_words(text):
    # Spliting the text using white space as delimiter beteen the words
    # Aufteilen des Textes mit Leerzeichen als Trennzeichen zwischen den Wörtern
    words = text.split()
    # Return the number of words
    # zeige die Anzahl der Wörter
    return len(words)

# use the code on our text
# Verwenden Sie den Code für unseren Text
word_count = count_words(PDF_file_text)
print(f"Number of words is: {word_count}")

Number of words is: 7523


In [6]:
# Building a function that split the text into chunks
# Erstellen einer Funktion, die den Text in Blöcke aufteilt

# i will adjust the chunk size to a 550 words, as i tried bigger but it didn't work
# Ich werde die Blockgröße auf 550 Wörter anpassen, da ich es mit einer größeren probiert habe, aber es hat nicht funktioniert
def text_split(text, chunk_size=550):
    # splitting the words in the text
    # Aufteilen der Wörter im Text
    words = text.split()
    # Create an empty list for chunks adding the words to the chunks
    # Erstelle eine leere Liste für Chunks und füge die Wörter zu den Chunks hinzu
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(' '.join(words[i:i + chunk_size]))
    return chunks

In [7]:
# I will use transformer from the "Hugging face" library, as it can offers High quality and state of art summarization
# First i will Define the function to summarize texts using Hugging Face Transformers
# adjust the min and max size words of the summarization and the chunks size
# Ich werde den Transformer aus der Bibliothek „Hugging Face“ verwenden, da dieser eine Zusammenfassung in hoher Qualität und auf dem neuesten Stand der Technik bietet.
# Zuerst werde ich die Funktion zum Zusammenfassen von Texten mithilfe von Hugging Face Transformers definieren.
# Passe die minimale und maximale Wortgröße der Zusammenfassung und die Chunk-Größe an.
def summarize(text, max_length=200, min_length=100, chunk_size=550):
    # i can try different models here, but i will choose "t5_base" because it have more parameters which make better performance
    # ich kann hier verschiedene Modelle ausprobieren, aber ich werde „t5 base“ wählen, weil es mehr Parameter hat, die für eine bessere Leistung bringen.
    summarization_pipeline = pipeline("summarization", model="t5-base")
    # split the text into chunks using the function we defined earlier
    # Teilen Sie den Text mit der zuvor definierten Funktion in Blöcke auf
    chunks = text_split(text, chunk_size=chunk_size)
    # summarizing the chunks starting with creating a list of summaries
    # Zusammenfassen der Blöcke, beginnend mit dem Erstellen einer Liste von Zusammenfassungen
    summaries = []
    for chunk in chunks:
        # summarization pipeline to generate a summary for each chunk
        # Zusammenfassungspipeline zum Generieren einer Zusammenfassung für jeden Block
        summary = summarization_pipeline(chunk, max_length=max_length, min_length=min_length, truncation=True)
        # adding each chunk summary to the summaries list we made
        # Hinzufügen jeder Blockzusammenfassung zur von uns erstellten Zusammenfassungsliste
        summaries.append(summary[0]['summary_text'])

    return ' '.join(summaries)

In [8]:
# Summarize the extracted text
# Fassen Sie den extrahierten Text zusammen
PDF_summary = summarize(PDF_file_text)
print("\nThe extracted Summary: \n", PDF_summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



The extracted Summary: 
 cultural studies of fashion have begun to articulate the variety of questions one can ask about fashion . is it a distinctively modern phenomenon or similar to traditionial uses of dress? fashion is a tangential concern to most academic disciplines; outside of historical costume studies there has been little sustained attention to the topic . interdisciplinary cultural studies have brought new life to topic by looking beyond familiar notions that fashion is manifestation of marketplace, social stratification, erotic drives, or cultural zeitgeists. fashion is a distinctively modern phenomenon that is fundamentally different from earlier uses of dress . early fashion theory contrasted the modern emphasis on change with the perceived stability of dress in premodern societies . fashion is defined as a system of conventions that requires both constant innovation and individual decision-making . if fashion is specific to modern consumer culture, it should not be see

Highlight keyphrases

In [9]:
# Finally, i will try to highlight the keyphrases in the text to make it easier to be read
# first i will import some needed libraries for the this task, libraries that help in identifiying stopwords, splitting the text and labeling the words

# Endlich, werde ich versuchen, die Schlüsselphrasen im Text hervorzuheben, um ihn leichter lesbar zu machen
# zuerst werde ich einige für diese Aufgabe benötigte Bibliotheken importieren, Bibliotheken, die dabei helfen, Stoppwörter zu identifizieren, den Text aufzuteilen und die Wörter zu kennzeichnen

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
# and then upload a library that display objects in the jupyter notebook and also if was in HTML format.
# und laden Sie dann eine Bibliothek hoch, die Objekte im Jupyter-Notebook anzeigt, auch wenn sie im HTML-Format wäre.
from IPython.core.display import display, HTML

In [21]:
# building the function that do the highlighting, and i will try different number of top keyphrases to be highlighted
# Erstellen der Funktion zum Hervorheben
def highlighting_keyphrases(text):
    # load the the stopwords, POS and word tokenization in the function
    # Laden Sie die Stoppwörter, POS und Wort-Tokenisierung in der Funktion
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    tagged = pos_tag(words)
    # iterate through the words
    # durch die Wörter iterieren
    key_phrases = [word for word, tag in tagged if tag.startswith('NN') and word.lower() not in stop_words]
    # creating a version of the text and replacing the phrases with HTML marked version of the phrase
    # Erstellen einer Version des Textes und Ersetzen der Phrasen durch eine HTML-markierte Version der Phrase
    highlighted_PDF = text
    for phrase in key_phrases:
        highlighted_PDF = highlighted_PDF.replace(phrase, f"<mark>{phrase}</mark>")
    # display the text within the jupyter notebook and then show the results
    # den Text im Jupyter-Notebook anzeigen und dann die Ergebnisse anzeigen
    display(HTML(highlighted_PDF))

highlighting_keyphrases(PDF_summary)

Discussion:

the code was successful, and for later recommendations, we can try different models for summarization and perform other visualizaiton methods on the extracted summary, like sensitive analysis or word cloud.

Diskussion:

Der Code war erfolgreich und für spätere Empfehlungen können wir verschiedene Modelle zur Zusammenfassung ausprobieren und andere Visualisierungsmethoden auf die extrahierte Zusammenfassung anwenden, wie etwa eine sensible Analyse oder eine Wortwolke.

The pdf was taken from this website:
https://www.researchgate.net/publication/316404875_Fashion